In [ ]:
# Load libraries
library(GEOquery)
library(SRAdb)
library(dplyr)
library(readr)

In [5]:
# Create directories for data storage
main_dir <- "NAFLD_NASH_RNAseq"
if (!dir.exists(main_dir)) {
  dir.create(main_dir)
}

metadata_dir <- file.path(main_dir, "metadata")
if (!dir.exists(metadata_dir)) {
  dir.create(metadata_dir)
}

raw_data_dir <- file.path(main_dir, "raw_data")
if (!dir.exists(raw_data_dir)) {
  dir.create(raw_data_dir)
}


In [ ]:
geo_accessions <- c(
  "GSE135251", # Liu et al. (2020)
  "GSE130970", # Govaere et al. (2020) - Transcriptomic Profiling Across NAFLD Spectrum
  "GSE126848", # Suppli et al. (2019)
)

In [14]:
# Function to download and process GEO dataset
download_geo_dataset <- function(geo_accession) {
  cat("Processing", geo_accession, "...\n")
  
  # Set output directory for this dataset
  dataset_dir <- file.path(raw_data_dir, geo_accession)
  if (!dir.exists(dataset_dir)) {
    dir.create(dataset_dir)
  }
  
  # Download GEO metadata
  gse <- getGEO(geo_accession, GSEMatrix = TRUE, getGPL = FALSE)
  
  # Extract metadata - fixed to handle list objects properly
  if (is(gse, "list")) {
    # If gse is a list (even with just one element), extract the first element
    metadata <- pData(phenoData(gse[[1]]))
  } else {
    # If gse is not a list but a direct ExpressionSet object
    metadata <- pData(phenoData(gse))
  }
  
  # Save metadata
  metadata_file <- file.path(metadata_dir, paste0(geo_accession, "_metadata.csv"))
  write.csv(metadata, metadata_file, row.names = FALSE)
  
  # Try to get SRA information if available
  sra_info <- NULL
  try({
    # Create or connect to the SRAdb SQLite database
    sra_dbname <- file.path(main_dir, "SRAmetadb.sqlite")
    if (!file.exists(sra_dbname)) {
      sra_dbname <- getSRAdbFile(destdir = main_dir)
    }
    con <- dbConnect(SQLite(), sra_dbname)
    
    # Find SRX IDs from the metadata (if available)
    srx_pattern <- "SRX[0-9]+"
    source_name_col <- which(grepl("source_name", colnames(metadata), ignore.case = TRUE))
    if (length(source_name_col) > 0) {
      potential_srx <- unlist(regmatches(metadata[, source_name_col], 
                                        gregexpr(srx_pattern, metadata[, source_name_col])))
    } else {
      # Look through all columns for SRX patterns
      all_text <- apply(metadata, 2, paste, collapse = " ")
      potential_srx <- unlist(regmatches(all_text, gregexpr(srx_pattern, all_text)))
    }
    
    if (length(potential_srx) > 0) {
      # Get SRA run information
      sra_info <- listSRAfile(search_terms = paste(potential_srx, collapse = " OR "), 
                              con = con, sraType = "sra")
    }
    
    # Close the SRA database connection
    dbDisconnect(con)
  }, silent = TRUE)
  
  # Extract expression data if available in GEO - fixed to handle list objects properly
  if (length(gse) > 0) {
    if (is(gse, "list")) {
      expr_data <- exprs(gse[[1]])
    } else {
      expr_data <- exprs(gse)
    }
    expr_file <- file.path(dataset_dir, paste0(geo_accession, "_expression_matrix.csv"))
    write.csv(expr_data, expr_file)
  }
  
  # Return information about the dataset
  return(list(
    accession = geo_accession,
    sample_count = nrow(metadata),
    sra_info = sra_info
  ))
}

In [ ]:
# Function to extract and save phenotype data for DESeq2 analysis
extract_phenotype_data <- function(geo_accession) {
  cat("Extracting phenotype data for", geo_accession, "...\n")
  
  # Load metadata
  metadata_file <- file.path(metadata_dir, paste0(geo_accession, "_metadata.csv"))
  if (!file.exists(metadata_file)) {
    cat("Metadata file not found for", geo_accession, "\n")
    return(NULL)
  }
  
  metadata <- read.csv(metadata_file)
  
  # Try to identify condition column
  condition_cols <- grep("disease|condition|state|status|source|nafld|nash|steatosis|fibrosis|characteristic", 
                         colnames(metadata), ignore.case = TRUE)
  
  if (length(condition_cols) == 0) {
    cat("No condition column found for", geo_accession, "\n")
    return(NULL)
  }
  
  # Create simplified phenotype data for DESeq2
  phenotype_data <- data.frame(
    sample_id = rownames(metadata)
  )
  
  # Add condition columns
  for (col in condition_cols) {
    col_name <- colnames(metadata)[col]
    phenotype_data[[col_name]] <- metadata[, col]
  }
  
  # Save phenotype data
  phenotype_file <- file.path(metadata_dir, paste0(geo_accession, "_phenotype.csv"))
  write.csv(phenotype_data, phenotype_file, row.names = FALSE)
  
  return(phenotype_data)
}

In [ ]:
# Process all GEO datasets
results <- list()
for (geo_accession in geo_accessions) {
  tryCatch({
    results[[geo_accession]] <- download_geo_dataset(geo_accession)
    extract_phenotype_data(geo_accession)
  }, error = function(e) {
    cat("Error processing", geo_accession, ":", conditionMessage(e), "\n")
    results[[geo_accession]] <- list(
      accession = geo_accession,
      sample_count = 0,
      sra_info = NULL,
      error = conditionMessage(e)
    )
  })
}

Processing GSE135251 ...


Found 1 file(s)

GSE135251_series_matrix.txt.gz

Using locally cached version: /scratch/RtmpsGjXZX/GSE135251_series_matrix.txt.gz



Extracting phenotype data for GSE135251 ...
Processing GSE130970 ...


Found 1 file(s)

GSE130970_series_matrix.txt.gz

Using locally cached version: /scratch/RtmpsGjXZX/GSE130970_series_matrix.txt.gz



Extracting phenotype data for GSE130970 ...
Processing GSE126848 ...


...

Extracting phenotype data for GSE49541 ...
Processing GSE151158 ...


Found 1 file(s)

GSE151158_series_matrix.txt.gz

Using locally cached version: /scratch/RtmpsGjXZX/GSE151158_series_matrix.txt.gz



Extracting phenotype data for GSE151158 ...
Processing GSE123876 ...


Found 1 file(s)

GSE123876_series_matrix.txt.gz

Using locally cached version: /scratch/RtmpsGjXZX/GSE123876_series_matrix.txt.gz



Extracting phenotype data for GSE123876 ...


In [ ]:
# Create a summary of downloaded datasets
summary_df <- data.frame(
  GEO_Accession = sapply(results, function(x) x$accession),
  Sample_Count = sapply(results, function(x) x$sample_count),
  SRA_Available = sapply(results, function(x) !is.null(x$sra_info)),
  SRA_Run_Count = sapply(results, function(x) ifelse(is.null(x$sra_info), 0, nrow(x$sra_info))),
  Error = sapply(results, function(x) ifelse(is.null(x$error), "", x$error))
)

summary_file <- file.path(main_dir, "dataset_summary.csv")
write.csv(summary_df, summary_file, row.names = FALSE)


In [21]:
# Function to download FASTQ files for a specific dataset
download_fastq_files <- function(geo_accession, max_samples = 2) {
  cat("Downloading FASTQ files for", geo_accession, "...\n")
  
  # Connect to SRA database
  sra_dbname <- file.path(main_dir, "SRAmetadb.sqlite")
  if (!file.exists(sra_dbname)) {
    sra_dbname <- getSRAdbFile(destdir = main_dir)
  }
  con <- dbConnect(SQLite(), sra_dbname)
  
  # Get SRA info for this GEO accession
  sra_info <- getSRAinfo(search_terms = geo_accession, con = con, sraType = "sra")
  
  # Limit to max_samples for demo purposes
  if (nrow(sra_info) > max_samples) {
    sra_info <- sra_info[1:max_samples, ]
  }
  
  # Download FASTQ files
  if (nrow(sra_info) > 0) {
    output_dir <- file.path(raw_data_dir, geo_accession)
    if (!dir.exists(output_dir)) {
      dir.create(output_dir)
    }
    
    # Get download URLs
    fastq_urls <- getFASTQinfo(sra_info$run, con = con)
    
    # Download files
    for (i in 1:nrow(fastq_urls)) {
      url <- fastq_urls$ftpURL[i]
      filename <- basename(url)
      output_file <- file.path(output_dir, filename)
      
      if (!file.exists(output_file)) {
        try({
          cat("Downloading", filename, "...\n")
          download.file(url, output_file)
        }, silent = TRUE)
      }
    }
  }
  
  # Close SRA database connection
  dbDisconnect(con)
}

In [22]:
# This will download FASTQ files for all studies listed in geo_accessions
for (geo_accession in geo_accessions) {
    download_fastq_files(geo_accession)
}

In [ ]:
# Function to perform basic DESeq2 analysis on one dataset as an example
run_deseq2_analysis <- function(geo_accession) {
  cat("Performing DESeq2 analysis on", geo_accession, "...\n")
  
  # Check for required packages
  if (!requireNamespace("DESeq2", quietly = TRUE)) {
    BiocManager::install("DESeq2")
  }
  library(DESeq2)
  
  # Load expression matrix
  expr_file <- file.path(raw_data_dir, geo_accession, paste0(geo_accession, "_expression_matrix.csv"))
  if (!file.exists(expr_file)) {
    cat("Expression matrix not found for", geo_accession, "\n")
    return(NULL)
  }
  
  expr_data <- read.csv(expr_file, row.names = 1)
  
  # Load phenotype data
  phenotype_file <- file.path(metadata_dir, paste0(geo_accession, "_phenotype.csv"))
  if (!file.exists(phenotype_file)) {
    cat("Phenotype data not found for", geo_accession, "\n")
    return(NULL)
  }
  
  phenotype_data <- read.csv(phenotype_file)
  
  # Find a suitable condition column
  condition_cols <- grep("disease|condition|state|status|nafld|nash|steatosis|fibrosis", 
                         colnames(phenotype_data), ignore.case = TRUE)
  
  if (length(condition_cols) == 0) {
    cat("No suitable condition column found for DESeq2 analysis\n")
    return(NULL)
  }
  
  # Use the first suitable condition column
  condition_col <- condition_cols[1]
  condition_name <- colnames(phenotype_data)[condition_col]
  
  # Ensure sample IDs match between expression and phenotype data
  common_samples <- intersect(colnames(expr_data), phenotype_data$sample_id)
  if (length(common_samples) == 0) {
    cat("No matching samples between expression and phenotype data\n")
    return(NULL)
  }
  
  expr_data <- expr_data[, common_samples]
  phenotype_data <- phenotype_data[phenotype_data$sample_id %in% common_samples, ]
  
  # Create DESeq2 object
  dds <- DESeqDataSetFromMatrix(
    countData = round(expr_data),  # DESeq2 requires integer counts
    colData = phenotype_data,
    design = as.formula(paste("~", condition_name))
  )
  
  # Run DESeq2 analysis
  dds <- DESeq(dds)
  
  # Get results
  res <- results(dds)
  
  # Save results
  results_dir <- file.path(main_dir, "deseq2_results")
  if (!dir.exists(results_dir)) {
    dir.create(results_dir)
  }
  
  results_file <- file.path(results_dir, paste0(geo_accession, "_deseq2_results.csv"))
  write.csv(as.data.frame(res), results_file)
  
  return(res)
}

# Print summary information
cat("\nDownload Summary:\n")
print(summary_df)

cat("\nData and metadata have been downloaded to:", main_dir, "\n")
cat("To download FASTQ files, uncomment the download_fastq_files function call\n")
cat("Note: Downloading all FASTQ files would require significant storage space and time\n")

cat("\nTo run DESeq2 analysis on a specific dataset, use the run_deseq2_analysis function\n")
cat("Example: run_deseq2_analysis(\"GSE130970\")\n")

# Function to extract differentially expressed genes for all datasets
extract_degs_from_all_datasets <- function() {
  # Check for required packages
  if (!requireNamespace("DESeq2", quietly = TRUE)) {
    BiocManager::install("DESeq2")
  }
  library(DESeq2)
  
  # Create results directory
  results_dir <- file.path(main_dir, "deseq2_results")
  if (!dir.exists(results_dir)) {
    dir.create(results_dir)
  }
  
  # Create a combined dataframe for all DEGs
  all_degs <- data.frame(
    gene = character(),
    dataset = character(),
    log2FoldChange = numeric(),
    padj = numeric()
  )
  
  # Process each dataset
  for (geo_accession in geo_accessions) {
    cat("Processing DEGs for", geo_accession, "...\n")
    
    # Check if expression data exists
    expr_file <- file.path(raw_data_dir, geo_accession, paste0(geo_accession, "_expression_matrix.csv"))
    if (!file.exists(expr_file)) {
      cat("Expression matrix not found for", geo_accession, ", skipping...\n")
      next
    }
    
    # Check if phenotype data exists
    phenotype_file <- file.path(metadata_dir, paste0(geo_accession, "_phenotype.csv"))
    if (!file.exists(phenotype_file)) {
      cat("Phenotype data not found for", geo_accession, ", skipping...\n")
      next
    }
    
    # Try running DESeq2 analysis
    tryCatch({
      res <- run_deseq2_analysis(geo_accession)
      
      if (!is.null(res)) {
        # Extract significant DEGs (padj < 0.05, |log2FC| > 1)
        res_df <- as.data.frame(res)
        res_df$gene <- rownames(res_df)
        
        sig_degs <- res_df[!is.na(res_df$padj) & res_df$padj < 0.05 & abs(res_df$log2FoldChange) > 1, ]
        
        if (nrow(sig_degs) > 0) {
          # Add to combined dataframe
          dataset_degs <- data.frame(
            gene = sig_degs$gene,
            dataset = rep(geo_accession, nrow(sig_degs)),
            log2FoldChange = sig_degs$log2FoldChange,
            padj = sig_degs$padj
          )
          
          all_degs <- rbind(all_degs, dataset_degs)
        }
      }
    }, error = function(e) {
      cat("Error running DESeq2 analysis for", geo_accession, ":", conditionMessage(e), "\n")
    })
  }
  
  # Save combined DEGs
  all_degs_file <- file.path(results_dir, "all_significant_degs.csv")
  write.csv(all_degs, all_degs_file, row.names = FALSE)
  
  # Find common DEGs across datasets
  if (nrow(all_degs) > 0) {
    # Count occurrences of each gene
    gene_counts <- table(all_degs$gene)
    
    # Find genes that appear in multiple datasets
    common_genes <- names(gene_counts[gene_counts > 1])
    
    # Save common genes
    common_genes_file <- file.path(results_dir, "common_degs.csv")
    common_degs <- all_degs[all_degs$gene %in% common_genes, ]
    write.csv(common_degs, common_genes_file, row.names = FALSE)
    
    cat("\nFound", length(common_genes), "genes differentially expressed in multiple datasets\n")
  } else {
    cat("\nNo significant differentially expressed genes found\n")
  }
  
  return(all_degs)
}

# Function to create a visualizable report of the analysis
create_analysis_report <- function() {
  if (!requireNamespace("ggplot2", quietly = TRUE)) {
    install.packages("ggplot2")
  }
  if (!requireNamespace("pheatmap", quietly = TRUE)) {
    install.packages("pheatmap")
  }
  
  library(ggplot2)
  library(pheatmap)
  
  # Create report directory
  report_dir <- file.path(main_dir, "report")
  if (!dir.exists(report_dir)) {
    dir.create(report_dir)
  }
  
  # Check if DESeq2 results exist
  results_dir <- file.path(main_dir, "deseq2_results")
  all_degs_file <- file.path(results_dir, "all_significant_degs.csv")
  
  if (file.exists(all_degs_file)) {
    all_degs <- read.csv(all_degs_file)
    
    # Create plots
    
    # 1. Number of DEGs per dataset
    degs_per_dataset <- as.data.frame(table(all_degs$dataset))
    colnames(degs_per_dataset) <- c("Dataset", "DEGs_Count")
    
    p1 <- ggplot(degs_per_dataset, aes(x = Dataset, y = DEGs_Count)) +
      geom_bar(stat = "identity", fill = "steelblue") +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
      labs(title = "Number of DEGs per Dataset", x = "Dataset", y = "Count")
    
    ggsave(file.path(report_dir, "degs_per_dataset.pdf"), p1, width = 10, height = 6)
    
    # 2. Volcano plot for one dataset (use the first one with results)
    datasets_with_degs <- unique(all_degs$dataset)
    if (length(datasets_with_degs) > 0) {
      example_dataset <- datasets_with_degs[1]
      results_file <- file.path(results_dir, paste0(example_dataset, "_deseq2_results.csv"))
      
      if (file.exists(results_file)) {
        res_df <- read.csv(results_file)
        
        # Create volcano plot
        res_df$significant <- ifelse(!is.na(res_df$padj) & res_df$padj < 0.05 & abs(res_df$log2FoldChange) > 1, "Yes", "No")
        
        p2 <- ggplot(res_df, aes(x = log2FoldChange, y = -log10(padj), color = significant)) +
          geom_point(alpha = 0.6) +
          scale_color_manual(values = c("No" = "grey", "Yes" = "red")) +
          theme_minimal() +
          labs(title = paste("Volcano Plot -", example_dataset),
               x = "log2 Fold Change", y = "-log10 Adjusted p-value")
        
        ggsave(file.path(report_dir, "volcano_plot.pdf"), p2, width = 8, height = 6)
      }
    }
    
    # 3. Top common DEGs heatmap
    common_genes_file <- file.path(results_dir, "common_degs.csv")
    if (file.exists(common_genes_file)) {
      common_degs <- read.csv(common_genes_file)
      
      # Get top 50 common genes by frequency
      gene_counts <- table(common_degs$gene)
      top_genes <- names(sort(gene_counts, decreasing = TRUE)[1:min(50, length(gene_counts))])
      
      # Create a matrix of log2FC values for heatmap
      top_common_degs <- common_degs[common_degs$gene %in% top_genes, ]
      
      # Reshape data for heatmap
      heatmap_data <- reshape2::dcast(top_common_degs, gene ~ dataset, value.var = "log2FoldChange")
      rownames(heatmap_data) <- heatmap_data$gene
      heatmap_data$gene <- NULL
      
      # Replace NA values with 0
      heatmap_data[is.na(heatmap_data)] <- 0
      
      # Create heatmap
      pdf(file.path(report_dir, "top_common_degs_heatmap.pdf"), width = 12, height = 10)
      pheatmap(heatmap_data, 
               main = "Top Common DEGs across Datasets",
               cluster_rows = TRUE, 
               cluster_cols = TRUE,
               scale = "row",
               color = colorRampPalette(c("blue", "white", "red"))(100))
      dev.off()
    }
    
    # Create an HTML report
    html_report <- file.path(report_dir, "analysis_report.html")
    cat("<html>\n<head>\n<title>NAFLD/NASH RNA-seq Analysis Report</title>\n", file = html_report)
    cat("<style>body { font-family: Arial, sans-serif; margin: 20px; }\n", file = html_report, append = TRUE)
    cat("h1, h2 { color: #2c3e50; }\n", file = html_report, append = TRUE)
    cat("table { border-collapse: collapse; width: 100%; }\n", file = html_report, append = TRUE)
    cat("th, td { text-align: left; padding: 8px; border: 1px solid #ddd; }\n", file = html_report, append = TRUE)
    cat("tr:nth-child(even) { background-color: #f2f2f2; }\n", file = html_report, append = TRUE)
    cat("th { background-color: #4CAF50; color: white; }\n", file = html_report, append = TRUE)
    cat("</style>\n</head>\n<body>\n", file = html_report, append = TRUE)
    
    cat("<h1>NAFLD/NASH RNA-seq Analysis Report</h1>\n", file = html_report, append = TRUE)
    
    # Dataset summary
    cat("<h2>Dataset Summary</h2>\n", file = html_report, append = TRUE)
    cat("<table>\n<tr><th>GEO Accession</th><th>Sample Count</th><th>SRA Available</th><th>DEGs Count</th></tr>\n", file = html_report, append = TRUE)
    
    for (i in 1:nrow(summary_df)) {
      deg_count <- sum(all_degs$dataset == summary_df$GEO_Accession[i])
      cat(sprintf("<tr><td>%s</td><td>%d</td><td>%s</td><td>%d</td></tr>\n", 
                  summary_df$GEO_Accession[i], 
                  summary_df$Sample_Count[i],
                  ifelse(summary_df$SRA_Available[i], "Yes", "No"),
                  deg_count), file = html_report, append = TRUE)
    }
    
    cat("</table>\n", file = html_report, append = TRUE)
    
    # Plots
    cat("<h2>Analysis Plots</h2>\n", file = html_report, append = TRUE)
    cat("<p>The following plots have been generated:</p>\n", file = html_report, append = TRUE)
    cat("<ul>\n", file = html_report, append = TRUE)
    cat("<li><a href='degs_per_dataset.pdf'>Number of DEGs per Dataset</a></li>\n", file = html_report, append = TRUE)
    cat("<li><a href='volcano_plot.pdf'>Volcano Plot</a></li>\n", file = html_report, append = TRUE)
    cat("<li><a href='top_common_degs_heatmap.pdf'>Top Common DEGs Heatmap</a></li>\n", file = html_report, append = TRUE)
    cat("</ul>\n", file = html_report, append = TRUE)
    
    # Top common DEGs table
    if (file.exists(common_genes_file)) {
      common_degs <- read.csv(common_genes_file)
      gene_counts <- table(common_degs$gene)
      top_genes <- names(sort(gene_counts, decreasing = TRUE)[1:min(20, length(gene_counts))])
      
      cat("<h2>Top 20 Common Differentially Expressed Genes</h2>\n", file = html_report, append = TRUE)
      cat("<table>\n<tr><th>Gene</th><th>Occurrences</th><th>Average Log2FC</th></tr>\n", file = html_report, append = TRUE)
      
      for (gene in top_genes) {
        gene_data <- common_degs[common_degs$gene == gene, ]
        avg_log2fc <- mean(gene_data$log2FoldChange)
        cat(sprintf("<tr><td>%s</td><td>%d</td><td>%.2f</td></tr>\n", 
                    gene, length(gene_data$gene), avg_log2fc), file = html_report, append = TRUE)
      }
      
      cat("</table>\n", file = html_report, append = TRUE)
    }
    
    cat("</body>\n</html>", file = html_report, append = TRUE)
    
    cat("\nAnalysis report created in:", report_dir, "\n")
  } else {
    cat("\nNo DESeq2 results found. Please run the extract_degs_from_all_datasets function first.\n")
  }
}

# Function to export candidate genes for Cas13 library design
export_candidate_genes_for_cas13 <- function() {
  # Check if DESeq2 results exist
  results_dir <- file.path(main_dir, "deseq2_results")
  common_genes_file <- file.path(results_dir, "common_degs.csv")
  
  if (!file.exists(common_genes_file)) {
    cat("No common DEGs file found. Please run the extract_degs_from_all_datasets function first.\n")
    return(NULL)
  }
  
  common_degs <- read.csv(common_genes_file)
  
  # Create library design directory
  library_dir <- file.path(main_dir, "cas13_library")
  if (!dir.exists(library_dir)) {
    dir.create(library_dir)
  }
  
  # Get gene counts
  gene_counts <- table(common_degs$gene)
  gene_occurrence <- as.data.frame(gene_counts)
  colnames(gene_occurrence) <- c("gene", "occurrence")
  
  # Calculate average log2FC for each gene
  gene_stats <- aggregate(log2FoldChange ~ gene, data = common_degs, FUN = function(x) c(mean = mean(x), sd = sd(x)))
  gene_stats_df <- do.call(data.frame, gene_stats)
  colnames(gene_stats_df) <- c("gene", "mean_log2FC", "sd_log2FC")
  
  # Merge occurrence and stats
  candidate_genes <- merge(gene_occurrence, gene_stats_df, by = "gene")
  
  # Sort by occurrence and mean log2FC
  candidate_genes <- candidate_genes[order(-candidate_genes$occurrence, abs(candidate_genes$mean_log2FC), decreasing = TRUE), ]
  
  # Add rank
  candidate_genes$rank <- 1:nrow(candidate_genes)
  
  # Save candidate genes
  candidate_genes_file <- file.path(library_dir, "candidate_genes_for_cas13.csv")
  write.csv(candidate_genes, candidate_genes_file, row.names = FALSE)
  
  # Create a prioritized list (top 100 or all if less)
  n_candidates <- min(100, nrow(candidate_genes))
  prioritized_genes <- candidate_genes[1:n_candidates, ]
  
  prioritized_genes_file <- file.path(library_dir, "prioritized_genes_for_cas13.csv")
  write.csv(prioritized_genes, prioritized_genes_file, row.names = FALSE)
  
  cat("\nCandidate genes for Cas13 library design exported to:", library_dir, "\n")
  cat("Total candidate genes:", nrow(candidate_genes), "\n")
  cat("Prioritized genes (top", n_candidates, "):", "see", prioritized_genes_file, "\n")
  